In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree

rng = np.random.RandomState(0)

In [2]:
df = pd.read_csv('Laptop-Preise.csv', sep=';', decimal=',')
df = df[df.extern_Schnittstellen != 2300] # Ausreißer löschen
df = df.reset_index(drop=True) # Reset Index
df.drop(['Betriebssystem_OHNE', 'Betriebssystem_Mac', 'Marke_Dell'], axis=1) # Mac und Marke_Apple sind identische Merkmale
df_noPrice = df.drop('Preis', axis=1)
error_mse = pd.DataFrame(columns=['chance', 'LM', 'KNN_1', 'KNN_3', 'KNN_5'])
error_stdErr = pd.DataFrame(columns=['chance', 'LM', 'KNN_1', 'KNN_3', 'KNN_5'])

In [3]:
def del_ran(df, chance):
    rand_array = np.random.rand(df.shape[0])
    delete_entries = rand_array < chance
    keep_entries = rand_array >= chance
    return {'keep' : df.iloc[keep_entries], 'delete' : df.iloc[delete_entries]}

In [4]:
temp = del_ran(df_noPrice, 0.01)
# temp['keep']
# temp['delete']

In [18]:
k = temp['keep']
d = temp['delete']
t = d.iloc[0].values.reshape(1, -1)

print(len(t))
print(k.shape)

tree = KDTree(temp['keep'], leaf_size=5)              
dist, ind = tree.query(t, k=3)



print(dist)
print(ind[0])
# [641 501 201]

pd.DataFrame(t)
pd.DataFrame(k.at(ind[0]))

# for t in d.iterrows():
#     dist, ind = tree.query(t, k=3)
#     print(ind, dist)


1
(1026, 32)
[[128.00390619 128.20686409 129.68037631]]
[641 501 201]


TypeError: '_AtIndexer' object is not callable

In [6]:
tree = KDTree(df, leaf_size=5)              
dist, ind = tree.query(df[:1], k=5) # Bei den Queries wird der Index 1 als Startindex verwendet? --> Nein
print(ind)  # indices of 3 closest neighbors (Indizes bei 0 beginnend)
print(dist)

[[   0  281  227  115 1028]]
[[  0.          61.05644929 100.01499888 100.98955392 101.30429408]]


In [7]:
df[:1]

,Preis,Kerne,Takt_Prozessor,Arbeitsspeicher,Bild_Display,Speicher_Grafik,extern_Schnittstellen,HDD,SSD,Akku_Kapazitaet,...,Marke_Acer,Marke_Apple,Marke_Dell,Marke_Lenovo,Betriebssystem_Windows,Betriebssystem_Mac,Betriebssystem_DOS,Betriebssystem_LINUX,Betriebssystem_OHNE,Mikrofon_Sound_vorhanden
0,299.0,2,2600,8,15.6,0,6,0,256,41.0,...,0,0,0,0,0,0,0,0,1,1


In [8]:
df.iloc[ind[0]]


,Preis,Kerne,Takt_Prozessor,Arbeitsspeicher,Bild_Display,Speicher_Grafik,extern_Schnittstellen,HDD,SSD,Akku_Kapazitaet,...,Marke_Acer,Marke_Apple,Marke_Dell,Marke_Lenovo,Betriebssystem_Windows,Betriebssystem_Mac,Betriebssystem_DOS,Betriebssystem_LINUX,Betriebssystem_OHNE,Mikrofon_Sound_vorhanden
0,299.0,2,2600,8,15.6,0,6,0,256,41.0,...,0,0,0,0,0,0,0,0,1,1
281,349.0,2,2600,8,17.3,0,6,0,256,41.0,...,0,0,0,0,0,0,0,0,1,1
227,399.0,2,2600,8,15.6,0,6,0,256,41.0,...,0,0,0,0,1,0,0,0,0,0
115,399.0,2,2600,8,17.3,0,6,0,256,41.0,...,0,0,0,0,0,0,0,0,1,1
1028,399.0,2,2600,8,14.0,0,6,0,256,41.0,...,0,0,0,0,0,0,1,0,0,0
